In [1]:
from pyspark.sql import SparkSession
from graphframes import *
import sparknlp
from sparknlp.annotator import Tokenizer, PerceptronModel
from sparknlp.base import DocumentAssembler
from pyspark.ml import Pipeline
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.clustering import LDA
from pyspark.sql import types as T
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StringType
from pyspark.sql import functions as F
import itertools

spark = SparkSession \
        .builder \
        .appName("network") \
        .getOrCreate()

24/11/25 20:56:18 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
data = spark.read.csv("../1moral_data.csv", header= True).select(["id", "cleaned_text"])

In [3]:
english = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", 
    "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "cannot", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", 
    "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "it", 
    "its", "itself", "let", "me", "more", "most", "must", "my", "myself", "no", "nor", "not", "of", "off", "on", 
    "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "some", "such", 
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", 
    "through", "to", "too", "under", "until", "up", "very", "was", "we", "were", "what", "when", "where", "which", 
    "while", "who", "whom", "why", "with", "would", "you", "your", "yours", "yourself", "yourselves", "will", "ll", 
    "re", "ve", "d", "s", "m", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", 
    "s", "t", "u", "v", "w", "x", "y", "z", "many", "us", "ok", "hows", "ive", "ill", "im", "cant", "topics", "topic",
    "discuss", "thoughts", "yo", "thats", "whats", "lets", "nothing", "oh", "omg", 
         "things", "stuff", "yall", "haha", "yes", "no", "wo", "like", 'good', 
         'work', 'got', 'going', 'dont', 'really', 'want', 'make', 'think', 
         'know', 'feel', 'people', 'life', "getting", "lot" "great", "i", "me", 
         "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", 
        "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
        "they", "them", "their", "theirs","themselves", "what", "which", "who", 
        "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
        "were", "be", "been", "being", "have", "has", "had", "having", "do", 
        "does", "did", "doing", "will", "would", "should", "can", "could", "may",
        "might", "must", "shall", "ought", "about", "above", "across", "after", 
        "against", "along", "amid", "among", "around", "as", "at", "before", "behind",
        "below", "beneath", "beside", "between", "beyond", "but", "by", 
        "concerning", "considering", "despite", "down", "during", "except", "for",
        "from", "in", "inside", "into", "like", "near", "next", "notwithstanding",
        "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past",
        "regarding", "round", "since", "than", "through", "throughout", "till", 
        "to", "toward", "towards", "under", "underneath", "unlike", "until", "up",
        "upon", "versus", "via", "with", "within", "without", "cant", "cannot", 
        "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", 
        "havent", "hed", "hell", "hes", "howd", "howll", "hows", "id", "ill", 
        "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", "mustve", 
        "mustnt", "shant", "shed", "shell", "shes", "shouldve", "shouldnt", 
        "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", 
        "theyll", "theyre", "theyve", "wed", "well", "were", "weve", "werent", 
        "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", 
        "whens", "whered", "wherell", "wheres", "whichd", "whichll", "whichre", 
        "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", 
        "whys", "wont", "wouldve", "wouldnt", "youd", "youll", "youre", "youve",
        "f", "m", "because", "go", "lot", "get", "still", "way", "something", "much",
        "thing", "someone", "person", "anything", "goes", "ok", "so", "just", "mostly", 
        "put", "also", "lots", "yet", "ha", "etc", "even", "one", "bye", "take", "wasnt"]

time = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", 
        "sunday", "morning", "noon", "afternoon", "evening", "night", "midnight",
        "dawn", "dusk", "week", "weekend", "weekends","weekly", "today", 
        "yesterday", "tomorrow", "yesterdays", "todays", "mondays", "tuesdays",
        "wednesdays", "thursdays", "fridays", "saturdays", "sundays", "day",
        "everyday", "daily", "workday", 'time', 'month', 'year', 'pm', 'am', "ago",
        "year", "now"]

reddit = ["welcome", "hi", "hello", "sub", "reddit", "thanks", "thank", "maybe",
          "wo30", "mods", "mod", "moderators", "subreddit", "btw", "aw", "aww", 
          "aww", "hey", "hello", "join", "joined", "post", "rselfimprovement", "blah"]

topic_specific = ["self", "improvement", "change", "action",
    'change', 'start', 'goal', 'habit', 'new', 'old', 
    'care', 'world', 'everyone', 'love', 'u', 'right', 'mean', 'matter',
    'best', 'step', 'focus', 'hard', 'small',
    'bad', 'help', 'time', 'problem', 'issue', 'advice',
    'bit', 'experience', 'different',
    'point', 'situation', 'negative', 'control', 'positive',
    'use', 'question', 'idea', 'amp', 'medium', 'hour', 'day', 'minute',
    'aaaaloot', "selfimprovement", "_"]

stopwords = english + time + reddit + topic_specific

In [4]:
documentAssembler = DocumentAssembler()\
     .setInputCol("cleaned_text")\
     .setOutputCol('document')

In [5]:
tokenizer = Tokenizer() \
            .setInputCols(['document'])\
            .setOutputCol('tokenized')

In [6]:
normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized')

In [7]:
stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['normalized']) \
     .setOutputCol('cleaned') \
     .setStopWords(stopwords)

In [ ]:
lemmatizer = LemmatizerModel.load("../models/lemma_ewt_en_3.4.3_3.0_1651416655397/")\
      .setInputCols("cleaned")\
      .setOutputCol("words")

In [9]:
from sparknlp.base import Finisher

finisher = Finisher().setInputCols(['words'])

In [10]:
my_pipeline = Pipeline(
      stages = [
          documentAssembler,
          tokenizer,
          normalizer,
          stopwords_cleaner,
          lemmatizer,
          finisher
      ])

In [11]:
pipelineModel = my_pipeline.fit(data)
processed_data = pipelineModel.transform(data)
processed_data.persist()
processed_data.show(1)

+-----+--------------------+--------------------+
|   id|        cleaned_text|      finished_words|
+-----+--------------------+--------------------+
|iqimz|i created this si...|[create, site, se...|
+-----+--------------------+--------------------+
only showing top 1 row



In [12]:
#Apply TF-IDF filtering
tfizer = CountVectorizer(inputCol='finished_words', outputCol='tf_features', minDF=0.01, vocabSize=1000)
tf_model = tfizer.fit(processed_data)
tf_result = tf_model.transform(processed_data)
vocabulary = tf_model.vocabulary


idfizer = IDF(inputCol='tf_features', outputCol='tf_idf_features')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)

processed_data.unpersist()
tfidf_result.persist()

DataFrame[id: string, cleaned_text: string, finished_words: array<string>, tf_features: vector, tf_idf_features: vector]

In [13]:
tfidf_result.printSchema()

root
 |-- id: string (nullable = true)
 |-- cleaned_text: string (nullable = true)
 |-- finished_words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tf_features: vector (nullable = true)
 |-- tf_idf_features: vector (nullable = true)



Asked ChatGPT "how can I filter tokenized words from TF IDF?"

In [14]:
# Define a function to filter words by their TF-IDF score
# UDF to map indices to words using the vocabulary
def filter_tfidf(features, threshold=25, vocabulary=None):
    if features is not None:
        # Filter based on TF-IDF score and map indices to actual words
        return [vocabulary[features.indices[i]] for i in range(len(features.values)) if features.values[i] >= threshold]
    return []

# Register the UDF
filter_udf = udf(lambda features: filter_tfidf(features, threshold=25, vocabulary=vocabulary), ArrayType(StringType()))

# Apply the filtering function
df_filtered_tfidf = tfidf_result.withColumn("filtered_words_tfidf", filter_udf("tf_idf_features"))

In [15]:
df_filtered_tfidf.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|   id|        cleaned_text|      finished_words|         tf_features|     tf_idf_features|filtered_words_tfidf|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|iqimz|i created this si...|[create, site, se...|(1000,[4,49,75,15...|(1000,[4,49,75,15...|                  []|
|pfzt5|hello everyone  i...|[recently, take, ...|(1000,[7,11,19,23...|(1000,[7,11,19,23...|                  []|
|pk714|i grew up with bo...|[grow, body, dysm...|(1000,[3,4,5,6,7,...|(1000,[3,4,5,6,7,...|                  []|
|qu825|well to give ever...|[give, everybody,...|(1000,[1,8,10,16,...|(1000,[1,8,10,16,...|                  []|
|rbi6h|i beginning to th...|[begin, inferiori...|(1000,[0,2,4,6,7,...|(1000,[0,2,4,6,7,...|                  []|
|tpsp4|this might come a...|[come, rant, disj...|(1000,[0,1,2,3,7,...|(1000,[0,1,2,3,7,...|     

In [ ]:
tfidf_result.unpersist()

Create network - Asked chatgpt how to create all possible pair combinations between a list of words

In [16]:
import itertools
def generate_edges(tokens):
    return [list(pair) for pair in itertools.combinations(tokens, 2)]

generate_edges_udf = udf(generate_edges, ArrayType(ArrayType(StringType())))

In [17]:
df_edges = df_filtered_tfidf.withColumn("edges", generate_edges_udf(F.col("filtered_words_tfidf")))

In [19]:
df_flat_edges = df_edges.select(
    F.col("id"),
    F.explode(F.col("edges")).alias("edge")
)

In [20]:
edges_df = df_flat_edges.select(
    F.col("id"),
    F.col("edge")[0].alias("src"),
    F.col("edge")[1].alias("dst")
)

edges_df = edges_df
edges_df.write.mode("overwrite").csv("edges_network2")
edges_df.show(10)


+------+------+------+
|    id|   src|   dst|
+------+------+------+
|17ctnf|   kid|   fun|
|1lnr36| cause| brain|
|1lnr36| cause|stress|
|1lnr36| cause| smile|
|1lnr36| cause| field|
|1lnr36| brain|stress|
|1lnr36| brain| smile|
|1lnr36| brain| field|
|1lnr36|stress| smile|
|1lnr36|stress| field|
+------+------+------+
only showing top 10 rows



In [21]:
vertices_df = edges_df.select(F.col("src").alias("id")).union(edges_df.select(F.col("dst").alias("id"))).distinct()
vertices_df.write.mode("overwrite").csv("nodes_network2")

In [22]:
vertices_df.show(10)

+------------+
|          id|
+------------+
|       inner|
|      online|
|conversation|
|       often|
|  productive|
|     achieve|
|       watch|
|     explain|
|   character|
|        grow|
+------------+
only showing top 10 rows



In [23]:
network = GraphFrame(vertices_df, edges_df)
network

/software/spark-3.3.2-el8-x86_64/python/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


GraphFrame(v:[id: string], e:[src: string, dst: string ... 1 more field])

In [24]:
network.inDegrees.orderBy(F.col("inDegree"), ascending=False).show()

/software/spark-3.3.2-el8-x86_64/python/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+-------------+--------+
|           id|inDegree|
+-------------+--------+
|   individual|     117|
|       belief|     109|
|         rule|      75|
|knowledgeable|      70|
|  information|      69|
|   successful|      64|
|        ampxb|      62|
|     question|      61|
|      ability|      59|
|      success|      57|
|    happiness|      56|
|      succeed|      53|
|          dad|      53|
|       unique|      53|
|       pursue|      52|
|    emotional|      52|
|       desire|      51|
|         task|      50|
|     business|      47|
|    motivated|      47|
+-------------+--------+
only showing top 20 rows



In [25]:
results = network.pageRank(resetProbability=0.01, maxIter=20)
results.vertices.select("id", "pagerank").orderBy(F.col("pagerank"), ascending=False).show(50)

+-------------+------------------+
|           id|          pagerank|
+-------------+------------------+
|    regularly|15.460319733225699|
|      extreme|12.547538500509107|
|       member|11.849140802317338|
|       unique| 11.59687772077006|
|    determine| 10.31618598885699|
|         fire| 9.538834403651848|
|        death|7.7503217283148365|
|        quote| 7.216798789675523|
|        extra| 6.739809718415769|
|  development| 6.608371365117951|
|   particular| 6.100894315069684|
|   individual| 5.865006476139233|
|    conscious|5.6630243440763675|
|     kindness| 5.506858584973658|
|        image| 5.488105883173017|
|      address| 5.360595061267332|
|     internal| 5.210499253867584|
|      abusive| 5.187160527245134|
|  communicate| 4.980519222798485|
|          rid| 4.614553437967763|
|      succeed| 4.538124397698883|
|       sister| 4.061027125671454|
|       reward|  4.05178771762602|
|    behaviour|4.0247676114382225|
|       boring|3.9977672453342947|
|      concept|3.856